In [34]:
import pandas
import pickle
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import re
import string
import nltk 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import numpy
from sklearn.model_selection import train_test_split

In [35]:
file = "trump_insult_tweets_2014_to_2021.csv"
ogfile = "labeled_data.csv" 
df = pandas.read_csv(file)
ogdf = pandas.read_csv(ogfile)


In [36]:
for i in ogdf.index:
    txt = ogdf.loc[i]["tweet"]
    txt=re.sub(r'@[A-Z0-9a-z_:]+','',txt)#replace username-tags
    txt=re.sub(r'[RT]+','',txt)#replace RT-tags
    txt=re.sub('https?://[A-Za-z0-9./]+','',txt)#replace URLs
    ogdf.at[i,"tweet"]=txt

In [37]:
ogdf.head

<bound method NDFrame.head of        Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0               0      3            0                   0        3      2   
1               1      3            0                   3        0      1   
2               2      3            0                   3        0      1   
3               3      3            0                   2        1      1   
4               4      6            0                   6        0      1   
...           ...    ...          ...                 ...      ...    ...   
24778       25291      3            0                   2        1      1   
24779       25292      3            0                   1        2      2   
24780       25294      3            0                   3        0      1   
24781       25295      6            0                   6        0      1   
24782       25296      3            0                   0        3      2   

                                             

In [38]:
#lower case all words before any preprocessing
ogdf["tweet"] = ogdf["tweet"].str.lower()

#removing punctuation
punctuations_list = string.punctuation
def remove_punctuations(text):
    temp = str.maketrans('', '',punctuations_list)
    return text.translate(temp)
ogdf['tweet'] = ogdf['tweet'].apply(lambda x: remove_punctuations(x))
ogdf.tail()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
24778,25291,3,0,2,1,1,yous a muthafin lie 8220 right his l is tras...
24779,25292,3,0,1,2,2,youve gone and broke the wrong heart baby and ...
24780,25294,3,0,3,0,1,young buck wanna eat dat nigguh like i aint fu...
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies
24782,25296,3,0,0,3,2,uffled ntac eileen dahlia beautiful color co...


In [40]:
#remove words that add no value to the tweet
# https://www.geeksforgeeks.org/hate-speech-detection-using-deep-learning/
def remove_stopwords(text):
    stop_words = stopwords.words('english')
    
    imp_words = []
    #store the important words
    for word in str(text).split():
        if word not in stop_words:
            #it's recommended to lemmatize the word as well
            #splitting it into its root which will provide a more accurate measure of which words are hate speech
            lemmatizer =WordNetLemmatizer()
            lemmatizer.lemmatize(word)
            imp_words.append(word)
    output = " ".join(imp_words)
    return output
ogdf['tweet'] = ogdf['tweet'].apply(lambda text: remove_stopwords(text))
ogdf.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,woman shouldnt complain cleaning house amp man...
1,1,3,0,3,0,1,boy dats coldtyga dwn bad cuffin dat hoe 1st p...
2,2,3,0,3,0,1,dawg ever fuck bitch start cry confused shit
3,3,3,0,2,1,1,look like tranny
4,4,6,0,6,0,1,shit hear might true might faker bitch told ya...


In [41]:
ogdf["target"] = ogdf["class"].map({0: 1,
                                 1: 1,
                                 2: 0})
ogdf.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet,target
0,0,3,0,0,3,2,woman shouldnt complain cleaning house amp man...,0
1,1,3,0,3,0,1,boy dats coldtyga dwn bad cuffin dat hoe 1st p...,1
2,2,3,0,3,0,1,dawg ever fuck bitch start cry confused shit,1
3,3,3,0,2,1,1,look like tranny,1
4,4,6,0,6,0,1,shit hear might true might faker bitch told ya...,1


In [43]:
y = ogdf["target"]
X = ogdf["tweet"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train = df['tweet']

In [44]:
# Load model from disk and use it to make new predictions
loaded_model = pickle.load(open("fit_model.pkl", 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

# Load test dataset
X_train = df['tweet']
pred = fit_model.pkl.predict(X_train)
print(pred)

MemoryError: 